# <center>Keyword extractor and summarizer</center>

Ressources 
Unicode : https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string
Rake : https://github.com/vgrabovets/multi_rake

Keyword extractor perso = every word except stop_words -> lemmatizer to apply to keep only the root
Keyword extractor summa = words more pertinent -> warning with summary test (no Japon)...
Keyword extractor Rake = words more pertinent -> same as summa, no Japon...

In [63]:
#keywords= lower, strip, token, summa-keywords
from nltk.corpus import stopwords
stop_words = stopwords.words('french')
new_stop_words = ['plus','quels','quel','quelle','quelles','tout','toutes']
for word in new_stop_words:
    stop_words.append(word)
def keywordizer(path,encoding):
    import re
    text = open(path,'r',encoding=encoding)
    text = text.read()
    text = text.lower().strip()
    text = re.sub('(@\S+)|\s\s+|[^\w\s]',' ',text)
    from nltk.tokenize import word_tokenize
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    import numpy as np
    text = np.unique(text)
    return print(text)

In [64]:
keywordizer('resume_cuisine-jap.txt','utf-8')

['anecdotes' 'bento' 'cartes' 'comment' 'compose' 'coup' 'coutumes'
 'cuisine' 'découverte' 'emballer' 'entièrement' 'façonner' 'fréquents'
 'garnir' 'gestes' 'histoires' 'illustré' 'ingrédients' 'invitation'
 'izakaya' 'japonais' 'japonaise' 'japonaises' 'livre' 'miso' 'plats'
 'pose' 'questions' 'recette' 'recettes' 'repas' 'réponses' 'saveurs'
 'savoir' 'soupe' 'sushi' 'techniques' 'traditionnelle' 'ustensiles' 'œil']


In [19]:
text = open('resume_cuisine-jap.txt','r',encoding='utf-8')
text = text.read()
from nltk.corpus import stopwords
stop_words = stopwords.words('french')
text = text.split()
text = [word for word in text if word not in stop_words]
text = ' '.join(text)
from summa import keywords
keywords = keywords.keywords(text)
keywords.split()

['comment',
 'toutes',
 'recettes',
 'recette',
 'japonaise',
 'japonaises',
 'entièrement',
 'illustré',
 'tout',
 'questions',
 'ustensiles',
 'ingrédients']

In [15]:
from multi_rake import Rake
text = open('resume_cuisine-jap.txt','r',encoding='utf-8')
text = text.read()
from nltk.corpus import stopwords
stop_words = stopwords.words('french')
text = text.split()
text = [word for word in text if word not in stop_words]
text = ' '.join(text)
r = Rake(max_words=1)
keywords = r.apply(text)
print(keywords[:20])

[('recettes', 1.0), ('anecdotes', 1.0), ('histoires', 1.0), ('cartes', 1.0), ('gestes', 1.0), ('techniques', 1.0), ('ustensiles', 1.0), ('ingrédients', 1.0), ('plats', 1.0)]


In [8]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
def summarizer(path,encoding):
	docx = open(path,'r',encoding=encoding)
	docx = docx.read()
	parser = PlaintextParser.from_string(docx,Tokenizer("french"))
	lex_summarizer = LexRankSummarizer()
	summary = lex_summarizer(parser.document,3)
	summary_list = [str(sentence) for sentence in summary]
	result = ' '.join(summary_list)
	return result

In [9]:
summarizer('resume_cuisine-jap.txt','utf-8')

'\ufeffUn livre entièrement illustré pour tout savoir sur la cuisine japonaise ! Des recettes, des anecdotes, des histoires, des cartes, des gestes, des techniques, des ustensiles, des ingrédients… Une invitation à la découverte des coutumes et des saveurs japonaises !Comment façonner les sushi ? Quelle est la recette traditionnelle de la soupe miso ?'

In [65]:
# NLP Pkgs
import spacy 
import re
nlp = spacy.load('fr_core_news_sm')
# Pkgs for Normalizing Text
from spacy.lang.fr.stop_words import STOP_WORDS
from string import punctuation
# Import Heapq for Finding the Top N Sentences
from heapq import nlargest
from nltk.corpus import stopwords


def text_summarizer(path,encoding):
    raw_docx = open(path,'r',encoding=encoding)
    raw_docx = raw_docx.read()#.lower().strip()
    docx = nlp(raw_docx)
    stop_words = stopwords.words('french')
    # Build Word Frequency # word.text is tokenization in spacy
    word_frequencies = {}  
    for word in docx:
        if word.text not in stop_words:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    # Sentence Tokens
    sentence_list = [ sentence for sentence in docx.sents ]

    # Sentence Scores
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]


    summarized_sentences = nlargest(3, sentence_scores, key=sentence_scores.get)
    final_sentences = [ w.text for w in summarized_sentences ]
    summary = ' '.join(final_sentences)
    return summary

In [66]:
text_summarizer('resume_pluies.txt','utf-8')

"Partout, les eaux avaient monté, les rivières enflées, on consolidait les digues, on en bâtissait d'autres, plus hautes, mais qui se révélaient de nouveau insuffisantes. Le phénomène, inexplicable, échappait à toute logique, à toute prévision, à tout modèle, à toute saison. \n Sur le port, les réfugiés se battent pour prendre place dans les derniers bateaux, pris de panique, convaincus qu'il s’agit là du dernier espoir de s'en sortir."

In [68]:
text_summarizer('resume_cuisine-jap.txt','utf-8-sig')

"Des recettes, des anecdotes, des histoires, des cartes, des gestes, des techniques, des ustensiles, des ingrédients… En un coup d’œil, toutes les réponses aux questions que l'on se pose ! Un livre entièrement illustré pour tout savoir sur la cuisine japonaise !"